In [ ]:
# O objetivo aqui é estudar e encontrar soluções para lidar com a quantidade excessiva de NaN's do dataset.
# A figura abaixo mostra a porcentagem de NaN's no Dataset o qual queremos lidar (passe o mouse em cima!).

![alt-text](\Users\joao\Desktop\image.png "Porcentagem de NaN's das variáveis que queremos lidar")

In [ ]:
import os
import pandas as pd
import AnaliseExploratoria_functions
from AnaliseExploratoria_functions import histogram_and_stats

current_path = os.getcwd()
csv_path = (current_path + '/Data/campeonatos_futebol_atualizacao.csv')

In [ ]:
df = pd.read_csv(csv_path)
df

In [ ]:
# primeiro precisamos tirar os outliers do dataframe. Podemos identificar os outliers.
# Podemos identificar outliers com a função "histogram_and_stats" da biblioteca AnaliseExploratoria_functions

In [ ]:
print(df.columns) # todas as colunas do dataframe original

In [ ]:
# Vamos selecionar apenas as colunas com valores numéricos e exibir suas estatísticas

In [ ]:
for col in df.select_dtypes(include="number").columns:
    print(f'\033[1m{col}\033[0m')
    histogram_and_stats(df,col,plot=False)

In [ ]:
# Analisando os valores máximos na célula acima, podemos ter uma noção de onde temos outliers.
# Nas células abaixo, usaremos alguns métodos para verificar outliers.

# Fazendo um loop pelos dados acima do percentil 99.9
df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    p999 = df[col].quantile(0.999)
    outliers_col = df[df[col] > p999]

    print(f"\n📊 Coluna: {col}")
    print(f"🔎 Quantidade de outliers (> P999 = {p999:.2f}): {len(outliers_col)}")

    # Mostra os outliers da coluna (apenas as colunas relevantes)
    if not outliers_col.empty:
        display(outliers_col[[col]])
    else:
        print("✅ Nenhum outlier encontrado acima do P999.")


In [ ]:

# Pelo Métdo do Intervalo Quartil 

# O IQR (Intervalo Interquartil) é uma medida estatística que mostra a dispersão central dos dados, ignorando os valores extremos. Ele é muito usado para detectar outliers.
# - Q1 (1º quartil): valor que separa os 25% menores valores
# - Q3 (3º quartil): valor que separa os 75% menores valores
# - O IQR seria IQR = Q3-Q1

# Definimos o limite para outliers:
# Limite  = Q3 + 1.5 × IQR
# Qualquer valor fora desse intervalo é considerado outlier.

# Portanto, aplicando o métdo

df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    q1 = df[col].quantile(0.01)
    q3 = df[col].quantile(0.99)
    iqr = q3 - q1

    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]

    print(f"\n📊 Coluna: {col}")
    print(f"🔎 Outliers encontrados: {len(outliers)}")

    if not outliers.empty:
        print(outliers[[col]].head())  # Mostra só os primeiros outliers
    else:
        print("✅ Nenhum outlier detectado.")


In [ ]:
# Agora, pelo métoddo dos z-scores
# O z-score mede o quão distante (em número de desvios padrão) um valor está da média da sua distribuição:
from scipy.stats import zscore

z = (x−μ)/σ

In [ ]:
df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    z = zscore(df[col], nan_policy='omit')  # calcula z-score, ignorando NaNs
    z_series = pd.Series(z, index=df.index)
    outliers = df[z_series.abs() > 4]

    print(f'\nOutliers na coluna "{col}":')
    print(outliers[[col]])

In [ ]:
## Tendo aplicado esses métodos, conseguimos identificar alguns outliers. Os outliers estão melhores detalhados no arquivo outliers.txt
outliers = (218, 220, 423, 571, 819, 841, 842, 846, 859, 868, 871, 1257,
            1490, 1497, 2305, 2410, 2414, 2457, 2841, 3148, 3183, 3997,
            5150, 6837, 6869, 6911, 7001, 7003, 7004, 7005, 7771, 10651,
            10718, 10819, 10923, 11084, 11406, 11568, 12005, 12185, 13750,
            13812, 14283, 15532, 19611, 19616, 19622, 19652, 19655, 19827,
            21357, 22641, 22669, 23085, 23133, 24787, 24788, 24838, 24855)

df.iloc[list(outliers)]


In [53]:
# Vamos eliminar esses outliers.
df_sem_outliers = df.drop(df.index[list(outliers)])
df_sem_outliers

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Tiros-livres 1,Tiros-livres 2,Defesas difíceis 1,Defesas difíceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Position 1,Position 2
0,8.0,0.0,6.0,3.0,7.0,1.0,6.0,1.0,8.0,14.0,...,NaN,NaN,NaN,NaN,77.0,23.0,Glasgow Rangers,Livingston,4-3-3,4-3-3
1,0.0,2.0,0.0,2.0,0.0,4.0,3.0,4.0,19.0,14.0,...,NaN,NaN,NaN,NaN,44.0,56.0,Ross County,St.Johnstone,3-5-2,3-5-2
2,4.0,5.0,1.0,5.0,8.0,11.0,2.0,5.0,13.0,14.0,...,NaN,NaN,NaN,NaN,47.0,53.0,Dundee FC,St. Mirren,4-3-3,3-4-1-2
3,4.0,7.0,8.0,1.0,6.0,5.0,4.0,7.0,4.0,11.0,...,NaN,NaN,NaN,NaN,27.0,73.0,Hearts,Celtic,3-4-3,4-2-3-1
4,3.0,1.0,1.0,3.0,5.0,4.0,2.0,2.0,12.0,17.0,...,NaN,NaN,NaN,NaN,58.0,42.0,Aberdeen,Dundee U.,4-4-2,3-5-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27711,6.0,2.0,1.0,0.0,9.0,6.0,6.0,5.0,7.0,7.0,...,NaN,NaN,NaN,NaN,52.0,48.0,Aston Villa,Blackburn,4-4-2,4-5-1
27712,6.0,0.0,4.0,3.0,4.0,2.0,8.0,2.0,11.0,9.0,...,NaN,NaN,NaN,NaN,52.0,48.0,Arsenal,Fulham,4-2-3-1,4-4-2
27713,7.0,7.0,2.0,3.0,10.0,2.0,8.0,0.0,10.0,6.0,...,NaN,NaN,NaN,NaN,55.0,45.0,Manchester Utd,Stoke,4-4-2,4-4-2
27714,6.0,6.0,1.0,2.0,4.0,3.0,4.0,6.0,5.0,3.0,...,NaN,NaN,NaN,NaN,45.0,55.0,West Ham,Manchester City,4-5-1,4-4-2


In [54]:
for col in df.select_dtypes(include="number").columns:
    print(f'\033[1m{col}\033[0m')
    histogram_and_stats(df,col,plot=False)

Chutes a gol 1
NaN's: 1512 --|-- Ocorrências: 26204 --|-- Média: 4.77 --|-- Desv. padrão: 2.80 --|-- Mínimo: 0.00 --|-- Máximo: 90.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 3.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 6.00 --|-- 95º percentil: 10.00
--------------
Chutes a gol 2
NaN's: 1512 --|-- Ocorrências: 26204 --|-- Média: 3.85 --|-- Desv. padrão: 2.44 --|-- Mínimo: 0.00 --|-- Máximo: 80.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 2.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 5.00 --|-- 95º percentil: 8.00
--------------
Impedimentos 1
NaN's: 2774 --|-- Ocorrências: 24942 --|-- Média: 2.14 --|-- Desv. padrão: 1.75 --|-- Mínimo: 0.00 --|-- Máximo: 23.00 --|-- 5º percentil: 0.00 --|-- 25º percentil: 1.00 --|-- 50º percentil (mediana): 2.00 --|-- 75º percentil: 3.00 --|-- 95º percentil: 5.00
--------------
Impedimentos 2
NaN's: 2774 --|-- Ocorrências: 24942 --|-- Média: 1.97 --|-- Desv. padrão: 1.68 --|-- Mínimo: 0.00 --|-- Máximo: 15.00 -

In [60]:
# Vamos salvar esses dados sem os outliers.
df_sem_outliers.to_csv(current_path + '/Data/campeonatos_futebol_sem_outliers.csv',
                       index=False)